In [ ]:
!pip install geopy requests

In [ ]:
import requests
from geopy.geocoders import Nominatim
from google.colab import userdata
import ipywidgets as widgets
from IPython.display import display
from getpass import getpass

try:
    api_key = userdata.get("WEATHER_API_KEY")
except:
    api_key = getpass("Enter your OpenWeatherMap API key: ")

geolocator = Nominatim(user_agent="weather_app")

def get_coordinates(place):
    try:
        location = geolocator.geocode(place)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

def get_weather(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url).json()
    if response.get("cod") == 200:
        return {
            "condition": response["weather"][0]["main"].lower(),
            "description": response["weather"][0]["description"],
            "temperature": response["main"]["temp"] - 273.15,
            "humidity": response["main"]["humidity"],
            "wind_speed": response["wind"]["speed"],
            "precipitation": response["rain"].get("1h", 0) if "rain" in response else 0
        }
    return None

def get_precautions(weather):
    condition = weather["condition"]
    temp = weather["temperature"]
    wind_speed = weather["wind_speed"]
    precipitation = weather["precipitation"]

    precautions = []

    if "rain" in condition or precipitation > 0:
        precautions.append("Carry an umbrella or wear a raincoat.")
        precautions.append("Wear waterproof shoes to stay dry.")
    if "snow" in condition:
        precautions.append("Wear warm, insulated clothing and snow boots.")
        precautions.append("Drive carefully; roads may be slippery.")
    if "clear" in condition or "sun" in condition:
        precautions.append("Apply sunscreen to protect your skin.")
        precautions.append("Wear sunglasses and a hat for sun protection.")
    if "cloud" in condition:
        precautions.append("Carry a light jacket in case it gets cooler.")
    if "storm" in condition or "thunder" in condition:
        precautions.append("Avoid open fields and tall objects to stay safe from lightning.")
        precautions.append("Stay indoors if possible.")
    if "fog" in condition or "mist" in condition:
        precautions.append("Drive slowly and use fog lights if traveling.")
        precautions.append("Wear reflective clothing for visibility.")

    if temp > 30:
        precautions.append("Stay hydrated; drink plenty of water.")
        precautions.append("Wear light, breathable clothing.")
    elif temp < 10:
        precautions.append("Wear warm layers to stay comfortable.")
        precautions.append("Use a scarf or gloves for extra warmth.")

    if wind_speed > 10:
        precautions.append("Secure loose items if outdoors.")
        precautions.append("Avoid cycling or high-profile vehicles in strong winds.")

    if not precautions:
        precautions.append("No specific precautions; enjoy your day!")

    return precautions

place_input = widgets.Text(description="Place:", placeholder="Enter city name (e.g., Guntur)")
button = widgets.Button(description="Get Weather & Precautions")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        place = place_input.value.strip()
        if not place:
            print("Please enter a place name.")
            return

        lat, lon = get_coordinates(place)
        if lat is None or lon is None:
            print(f"Could not find coordinates for {place}. Please check the name.")
            return

        weather = get_weather(lat, lon)
        if weather is None:
            print("Error fetching weather data. Check API key or try again later.")
            return

        precautions = get_precautions(weather)

        print(f"Current Weather in {place}:")
        print(f"Condition: {weather['description'].capitalize()}")
        print(f"Temperature: {weather['temperature']:.1f}°C")
        print(f"Humidity: {weather['humidity']}%")
        print(f"Wind Speed: {weather['wind_speed']} m/s")
        print("\nPrecautionary Measures:")
        for precaution in precautions:
            print(f"- {precaution}")

button.on_click(on_button_clicked)
display(place_input, button, output)

Enter your OpenWeatherMap API key: ··········


Text(value='', description='Place:', placeholder='Enter city name (e.g., Guntur)')

Button(description='Get Weather & Precautions', style=ButtonStyle())

Output()